<p style="font-family: Arial; font-size:3.75em; color:purple; font-style:bold">
<br>BLOBS CHARACTERIZER</p><br>

This notebook characterizes the BLOBs. It's main pourpouse is to identify what is the best radius and energy threshold to be used as parameters in the classical "blobs cut".
It is done in two different steps:
> * Identify wich is the minimum blob radius that performs a good discrimination between blobs of single and double electron tracks.
> * Once identified the blob radius, determine wich is the energy threshold that gives a better figure of merit: S/sqrt(N)

In [ ]:
%matplotlib inline
%load_ext memory_profiler
%load_ext line_profiler

%load_ext autoreload
%autoreload 2

In [ ]:
from IPython.core.display import HTML
css = open('style-table.css').read() + open('style-notebook.css').read()
HTML('<style>{}</style>'.format(css))

#### Importing general stuff

In [ ]:
import os
import sys
import numpy  as np
import tables as tb
import pandas as pd
import matplotlib.pyplot as plt
from   matplotlib.colors import LogNorm

# Specific IC stuff
import invisible_cities.core.system_of_units  as units
from invisible_cities.evm.event_model         import Voxel
from invisible_cities.reco.paolina_functions  import make_track_graphs
from invisible_cities.reco.paolina_functions  import blob_energies

# Specific fanalIC stuff
from fanal.reco.reco_io_functions import get_reco_group_name
from fanal.ana.ana_io_functions   import get_ana_group_name

from fanal.ana.ana_functions import get_new_energies
from fanal.ana.ana_functions import process_tracks

#### GENERAL ANALYSIS DATA 

In [ ]:
# DETECTOR NAME
DET_NAME = 'next100'

# BASE PATH
BASE_PATH = '/Users/Javi/Development/FANAL/data'

# VERBOSITY
VERBOSE = True

# ENERGY RESOLUTION
fwhm = 0.5

# VOXEL SIZE
voxel_size = [3, 3, 3]

# Track energy threshold
track_Eth = 3. * units.keV

In [ ]:
# INPUT DIRS
fwhm_str   = str(fwhm).replace('.', '')
voxel_str  = f"{voxel_size[0]}x{voxel_size[0]}x{voxel_size[0]}"

bb0nu_fname = f"{BASE_PATH}/{DET_NAME}/bb0nu/reco/{DET_NAME}.bb0nu.fwhm_{fwhm_str}.voxel_{voxel_str}.reco.h5"
Bi214_fname = f"{BASE_PATH}/{DET_NAME}/Bi214/reco/{DET_NAME}.Bi214.fwhm_{fwhm_str}.voxel_{voxel_str}.reco.h5"
Tl208_fname = f"{BASE_PATH}/{DET_NAME}/Tl208/reco/{DET_NAME}.Tl208.fwhm_{fwhm_str}.voxel_{voxel_str}.reco.h5"

In [ ]:
reco_group_name = get_reco_group_name(fwhm, voxel_size)

# SELECTING the blob radius

To do it, we will plot the evolution of the Blob2 energy in terms of the radius for signal and background, and we will select the minimum radius in which these energy distributions are clearly different.

#### Range of blob-radius to study

In [ ]:
MIN_RAD = 5
MAX_RAD = 25
blob_radius = np.arange(MIN_RAD, MAX_RAD)

#### Loading into memory all the input data

In [ ]:
def load_data(file_names):
    events = pd.DataFrame()
    voxels = pd.DataFrame()

    # Looping through all the input files
    for iFileName in file_names:
        file_events = pd.read_hdf(iFileName, reco_group_name + '/events')
        file_voxels = pd.read_hdf(iFileName, reco_group_name + '/voxels')
    
        events = pd.concat([events, file_events], axis=0)
        voxels = pd.concat([voxels, file_voxels], axis=0)
    
    return events, voxels


In [ ]:
bb0nu_events, bb0nu_voxels = load_data([bb0nu_fname])
Bi214_events, Bi214_voxels = load_data([Bi214_fname])
Tl208_events, Tl208_voxels = load_data([Tl208_fname])

#### Getting IC voxels from events with just one track

In [ ]:
def get_ic_voxels(events, voxels):
    '''
    For every fiducial event, tracks are made with the voxels.
    From those with just one reconstructed tracks, it returns
    the collection oc IC voxels to play the blobs game.
    '''
    events_ic_voxels = []

    ### Looping through all the events that passed the fiducial filter
    ### and getting the voxels
    for event_id, event_df in events[events.fid_filter].iterrows():
        event_voxels     = voxels.loc[event_id]
        voxel_dimensions = (event_df.voxel_sizeX, event_df.voxel_sizeY, event_df.voxel_sizeZ)

        # If there is any negligible Voxel, distribute its energy between its neighbours,
        # if not, all voxels maintain their previous energies
        if len(event_voxels[event_voxels.negli == True]):
            event_voxels_newE = get_new_energies(event_voxels)
        else:
            event_voxels_newE = event_voxels.E.tolist()
            
        # Translate fanalIC voxels info to IC voxels to make tracks
        ic_voxels = []
        for i, voxel in event_voxels.iterrows():
            ic_voxel = Voxel(voxel.X, voxel.Y, voxel.Z, event_voxels_newE[i], voxel_dimensions)
            ic_voxels.append(ic_voxel)            
        
        # Make and process tracks
        event_tracks        = make_track_graphs(ic_voxels)
        event_sorted_tracks = process_tracks(event_tracks, track_Eth)         
        num_tracks = len(event_sorted_tracks)
        
        if(num_tracks == 1):
            events_ic_voxels.append(event_sorted_tracks[0][1])
        
    return events_ic_voxels
    

In [ ]:
bb0nu_ic_voxels = get_ic_voxels(bb0nu_events, bb0nu_voxels)

In [ ]:
Bi214_ic_voxels = get_ic_voxels(Bi214_events, Bi214_voxels)

In [ ]:
Tl208_ic_voxels = get_ic_voxels(Tl208_events, Tl208_voxels)

In [ ]:
bb0nu_ic_voxels

#### Getting Blob2 energies for all the range of Blob radius

In [ ]:
signal_blob2E_dict = {}
bkgnd_blob2E_dict  = {}

# For every radius to study
for rad in blob_radius:
    rad_str = f"{rad}mm"
    
    # Going through all the voxel sets from signal events
    signal_blob2E = []
    for voxel_set in bb0nu_ic_voxels:
        blob1_E, blob2_E = blob_energies(voxel_set, rad * units.mm)
        signal_blob2E.append(blob2_E)
    signal_blob2E_dict[rad_str] = signal_blob2E

    # Going through all the voxel sets from background events
    bkgnd_blob2E = []
    for voxel_set in Tl208_ic_voxels:
        blob1_E, blob2_E = blob_energies(voxel_set, rad * units.mm)
        bkgnd_blob2E.append(blob2_E)
    for voxel_set in Bi214_ic_voxels:
        blob1_E, blob2_E = blob_energies(voxel_set, rad * units.mm)
        bkgnd_blob2E.append(blob2_E)
    bkgnd_blob2E_dict[rad_str] = bkgnd_blob2E

signal_blob2E = pd.DataFrame(signal_blob2E_dict)
bkgnd_blob2E  = pd.DataFrame(bkgnd_blob2E_dict)


In [ ]:
#bkgnd_blob2E

In [ ]:
#signal_blob2E

#### Plotting 1

In [ ]:
for rad in blob_radius:
    rad_str = f"{rad}mm"

    fig = plt.figure(figsize = (15,5))
    num_bins = 50
    
    ax1 = fig.add_subplot(1, 2, 1)
    signal_blob2E[rad_str].hist(bins=num_bins, range=[0.,1.0])
    plt.xlabel('Blob2 Energy [MeV]')
    plt.title(f"Signal Blob2 Energy - Rad: {rad} mm", size=14)
    
    ax2 = fig.add_subplot(1, 2, 2)
    bkgnd_blob2E[rad_str].hist(bins=num_bins, range=[0.,1.0])
    plt.xlabel('Blob2 Energy [MeV]')
    plt.title(f"Background Blob2 Energy - Rad: {rad} mm", size=14)

    plt.show()

#### Plotting 2

In [ ]:
# PLOT DETAILS ...
min_rad_toPlot = 10
max_rad_toPlot = 25

max_E_toPlot     = 1.0 * units.MeV
num_Ebins_toPlot = 20

In [ ]:
# Generating data for hist2d plotting
signal_rad = []
signal_E   = []
bkgnd_rad  = []
bkgnd_E    = []

for rad in np.arange(min_rad_toPlot, max_rad_toPlot):
    rad_str = f"{rad}mm"

    # Signal data
    signal_rad += [rad] * len(signal_blob2E[f"{rad}mm"])
    signal_E   += list(signal_blob2E[f"{rad}mm"])

    # Background data
    bkgnd_rad += [rad] * len(bkgnd_blob2E[f"{rad}mm"])
    bkgnd_E   += list(bkgnd_blob2E[f"{rad}mm"])

# Generating bins
bins_y = list(np.arange(0, max_E_toPlot, (max_E_toPlot/num_Ebins_toPlot)))
bins_x = [min_rad_toPlot-0.5]
for rad in np.arange(min_rad_toPlot, max_rad_toPlot):
    bins_x += [rad + 0.5]

# Plotting
fig = plt.figure(figsize = (18,6));
ax1 = fig.add_subplot(1, 2, 1)
plt.hist2d(signal_rad, signal_E, bins=(bins_x, bins_y))
plt.xlabel("Blob Radius [mm]")
plt.ylabel("Blob2 E [MeV]")
plt.title("Signal Blob2 Energy vs Blob radius")
plt.colorbar(label='entries')

ax2 = fig.add_subplot(1, 2, 2)
plt.hist2d(bkgnd_rad, bkgnd_E, bins=(bins_x, bins_y))
plt.xlabel("Blob Radius [mm]")
plt.ylabel("Blob2 E [MeV]")
plt.title("Background Blob2 Energy vs Blob radius")
plt.colorbar(label='entries')


# SELECTING the blob energy threshold

Once selected the appropriate blob radius, let's select wich blob energy threshold yields the best figure of merit (signal / sqrt(bkgnd)).

In [ ]:
BLOB_RAD = 15 * units.mm

In [ ]:
max_E_toPlot     = 1.2 * units.MeV
num_Ebins_toPlot = 36

fig = plt.figure(figsize = (18,6))
ax1 = fig.add_subplot(1, 2, 1)
plt.hist(signal_blob2E[f"{int(BLOB_RAD)}mm"], num_Ebins_toPlot, [0,max_E_toPlot])
plt.title(f"Signal Blob2 Energy - Blob rad = {BLOB_RAD} mm")
plt.xlabel("Blob2 E [MeV]")
plt.ylabel("entries")

ax2 = fig.add_subplot(1, 2, 2)
plt.hist(bkgnd_blob2E[f"{int(BLOB_RAD)}mm"], num_Ebins_toPlot, [0,max_E_toPlot])
plt.title(f"Background Blob2 Energy - Blob rad = {BLOB_RAD} mm")
plt.xlabel("Blob2 E [MeV]")
plt.ylabel("entries")


In [ ]:
import math
energy_thresholds = np.arange(0.2, 0.8, 0.01)

tot_signal = len(signal_blob2E[f"{int(BLOB_RAD)}mm"])

merits = []
for Eth in energy_thresholds:
    signal = len(signal_blob2E[signal_blob2E[f"{int(BLOB_RAD)}mm"] > Eth])
    bkgnd  = len(bkgnd_blob2E[bkgnd_blob2E[f"{int(BLOB_RAD)}mm"] > Eth])
    if (bkgnd): merit = signal / math.sqrt(bkgnd)
    else:       merit = 0.
    merits.append(merit)
    print(f"Eth [MeV]: {Eth:5.3}  ->  Signal: {signal:5} ({signal/tot_signal*100:6.4}%)   Bkgnd: {bkgnd:5}   Merit: {merit:.4}")

fig = plt.figure(figsize = (10,8))
plt.plot(energy_thresholds, merits)
plt.title(f"Figure of Merit S/sqrt(N) - Blob rad = {BLOB_RAD} mm")
plt.xlabel("Blob Energy Threshold (MeV)")
plt.ylabel("Arbitrary Units")
